In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import (
    clsf_boz_cd_c, clsf_crd_grd, get_disc_factor_all_c, get_loss_adj_rate_all_c
)

In [2]:
# AT_BSC_CVR_CD : 01=대인1, 02=대인2, 03=대물, 04=자기신체/자동차상해, 05=무보험, 06=자기차량, 07=기계장치, 08=다른자동차운전, 09=SOS, 10=기본담보
# AT_PDGR_CD : 01=개인용, 02=업무용, 03=영업용, 04=이륜차, 05=외화표시, 06=운전자, 07=취급업자, 08=모터바이크, 09=농기계, 10=운전면허교습생, 11=회사단독
# AT_CHN_DVCD : 01=전통, 02=GA, 03=TM, 04=CM

In [3]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_자동차')
BASE_YYMM = '201912'

In [4]:
# 데이터 불러오기
## 보험료부채 익스포져
자동차_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str})
자동차_비비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_미경과보험료_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'SL_CVR_CD': str, 'T02_RN_RINSC_CD': str})
# 일반_원수_예정보험료 = pd.read_excel(FILE_PATH / f'일반_원수_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
# 일반_출재_예정보험료 = pd.read_excel(FILE_PATH / f'일반_출재_예정보험료_{BASE_YYMM}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
자동차_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx', dtype={'PDC_CD': str, 'T02_RN_RINSC_CD': str})

## 준비금부채 익스포져
자동차_원수_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_원수_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str})
자동차_비비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_개별추산액_{BASE_YYMM}.xlsx', dtype={'T02_RN_RINSC_CD': str, 'PDC_CD': str})

## 기초정보, 가정 등
# 자동차_코드맵핑 = pd.read_excel(FILE_PATH / '자동차_코드맵핑.xlsx', 
#     dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
# 자동차_코드맵핑.to_csv(FILE_PATH / '자동차_코드맵핑.csv', index=False)
자동차_코드맵핑 = pd.read_csv(FILE_PATH / '자동차_코드맵핑.csv',
    dtype={'PDC_CD': str, 'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_보험금진전추이 = pd.read_excel(FILE_PATH / f'자동차_보험금진전추이_{201812}.xlsx', dtype={'PDGR_CD': str, 'AY': str}) \
        .rename(columns={'AT_PDGR_CD': 'BOZ_CD', 'AY_YM': 'AY'})
자동차_최종손해율 = pd.read_excel(FILE_PATH / f'자동차_최종손해율_{201812}.xlsx', dtype={'RRNR_DVCD': str, 'AT_CHN_DVCD': str, 'AT_PDGR_CD': str}) \
    .drop('RRNR_DAT_DVCD', axis=1)
자동차_사업비율 = pd.read_excel(FILE_PATH / f'자동차_사업비율_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BOZ_CD': str})
자동차_IBNR = pd.read_excel(FILE_PATH / f'자동차_IBNR_{BASE_YYMM}.xlsx', dtype={'RRNR_DVCD': str, 'BSC_CVR_CD': str})

## 공통
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{202012}.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{202012}.xlsx')
할인율 = pd.read_excel(FILE_PATH / f'할인율_{202012}.xlsx')

In [5]:
# 데이터 전처리
## 미경과보험료
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료.copy()
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_원수_미경과보험료_가공['RRNR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_미경과보험료_가공, 자동차_상품군정보)
자동차_원수_미경과보험료_가공['CRD_GRD'] = '#'
자동차_원수_미경과보험료_가공.rename(columns={'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'CRD_GRD', 'URND_PRM']]

자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료.copy()
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비례출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_미경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_미경과보험료_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비례출재_미경과보험료_가공.rename(columns={'RN_LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'CRD_GRD', 'URND_PRM']]

자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료.copy()
자동차_비비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_미경과보험료_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_미경과보험료_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비비례출재_미경과보험료_가공['AT_PDGR_CD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공['AT_CHN_DVCD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공.rename(columns={'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'CRD_GRD', 'URND_PRM']]

자동차_미경과보험료_가공 = pd.concat([자동차_원수_미경과보험료_가공, 자동차_비례출재_미경과보험료_가공, 자동차_비비례출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비.copy()
자동차_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_원수_직전3년보험금손조비_가공['CRD_GRD'] = '#'
자동차_원수_직전3년보험금손조비_가공.rename(columns={'AT_DAG_IVMT': 'DAG_IVMT'}, inplace=True)
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비.copy()
자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비_가공.assign(RN_DAG_IVMT=lambda x: x['RN_DAG_IVMT'].fillna(0))
자동차_비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_비례출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_비례출재_직전3년보험금손조비_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)
자동차_비례출재_직전3년보험금손조비_가공.rename(columns={'RN_DAG_IVMT': 'DAG_IVMT', 'RN_PYN_BNF': 'PYN_BNF'}, inplace=True)
자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비.copy()
자동차_비비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_직전3년보험금손조비_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_직전3년보험금손조비_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_직전3년보험금손조비_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_직전3년보험금손조비_가공, 재보험자_국내신용등급)
자동차_비비례출재_직전3년보험금손조비_가공.rename(columns={'WNCR_CNV_DAG_INV_CS': 'DAG_IVMT', 'WNCR_CNV_DS_RBNF_NET': 'PYN_BNF'}, inplace=True)
자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'PYN_BNF', 'DAG_IVMT']]

자동차_직전3년보험금손조비_가공 = pd.concat([자동차_원수_직전3년보험금손조비_가공, 자동차_비례출재_직전3년보험금손조비_가공, 자동차_비비례출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

## 개별추산액
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액.copy()
자동차_원수_개별추산액_가공['RRNR_DVCD'] = '01'
자동차_원수_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_개별추산액_가공, 자동차_상품군정보)
자동차_원수_개별추산액_가공['CRD_GRD'] = '#'
자동차_원수_개별추산액_가공.rename(columns={'LTPD_OST_BNF': 'OST_AMT'}, inplace=True)
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_AMT']]

자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액.copy()
자동차_비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_비례출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_개별추산액_가공, 자동차_상품군정보)
자동차_비례출재_개별추산액_가공['T02_RN_RINSC_CD'] = '124012'
자동차_비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비례출재_개별추산액_가공.rename(columns={'RN_LTPD_OST_BNF': 'OST_AMT'}, inplace=True)
자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_AMT']]

자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액.copy()
자동차_비비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_개별추산액_가공['DMFR_DVCD'] = '01'
자동차_비비례출재_개별추산액_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비비례출재_개별추산액_가공.rename(columns={'WNCR_CNV_OST_RBNF': 'OST_AMT'}, inplace=True)
자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD', 'OST_AMT']]

자동차_개별추산액_가공 = pd.concat([자동차_원수_개별추산액_가공, 자동차_비례출재_개별추산액_가공, 자동차_비비례출재_개별추산액_가공], axis=0).reset_index(drop=True)

In [6]:
# 가정산출
## 할인요소
자동차_할인요소 = get_disc_factor_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'))

## 재보험자산 손실조정율
손실조정율 = get_loss_adj_rate_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'), 선도부도율)

## 손해조사비율
### 원수, 출재(비례+비비례)로 그룹핑
자동차_손해조사비율 = 자동차_직전3년보험금손조비_가공 \
    .groupby(['RRNR_DVCD', 'BOZ_CD'])[['PYN_BNF', 'DAG_IVMT']].sum() \
    .eval('LAE_RATIO = DAG_IVMT/PYN_BNF').reset_index() \
    [['RRNR_DVCD', 'BOZ_CD', 'LAE_RATIO']]

## IBNR
자동차_미경과보험료_소계 = 자동차_미경과보험료_가공 \
    .groupby(['RRNR_DVCD', 'BSC_CVR_CD'])['URND_PRM'].sum().reset_index(name='URND_PRM_TOT')
자동차_IBNR_가공 = 자동차_미경과보험료_가공.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'CRD_GRD'], as_index=False)['URND_PRM'].sum() \
    .merge(자동차_미경과보험료_소계, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .eval('URND_PRM_WGT = URND_PRM/URND_PRM_TOT') \
    .merge(자동차_IBNR, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .assign(KRW_OS_AMT_E = lambda x: x['KRW_OS_AMT_E'].fillna(0)) \
    .assign(IBNR = lambda x: x['KRW_OS_AMT_E']*x['URND_PRM_WGT']) \
    .groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'CRD_GRD'])['IBNR'].sum().reset_index()

In [23]:
# 데이터 집계
## 보험료부채
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'CRD_GRD']
자동차_미경과보험료_집계 = 자동차_미경과보험료_가공.groupby(KEYS)['URND_PRM'].sum().reset_index()
자동차_보험료부채 = 자동차_미경과보험료_집계 \
    .merge(자동차_최종손해율, on=['AT_PDGR_CD', 'AT_CHN_DVCD'], how='left') \
    .merge(자동차_손해조사비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .merge(자동차_사업비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .assign(EXP_RT = lambda x: x['EXP_RT'].fillna(0).astype(float)) \
    .merge(자동차_할인요소[['BOZ_CD', 'DISC_FAC_PRM']], on=['BOZ_CD'], how='left') \
    .merge(손실조정율[['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_PRM']], on=['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD'], how='left') \
    .assign(LOSS_ADJ_RATE_PRM = lambda x: x['LOSS_ADJ_RATE_PRM'].fillna(0).astype(float))
자동차_보험료부채 = 자동차_보험료부채.eval('EPCT_PRM = 0') # 삭제예정
자동차_보험료부채 = 자동차_보험료부채
자동차_보험료부채 = 자동차_보험료부채 \
        .eval('FNAL_LOSS = (URND_PRM+EPCT_PRM)*FNAL_LSRT') \
        .eval('LAE = FNAL_LOSS*LAE_RATIO') \
        .eval('MAINT_EXP = (URND_PRM+EPCT_PRM)*EXP_RT') \
        .eval('LIAB_PRM = -EPCT_PRM+FNAL_LOSS+LAE+MAINT_EXP') \
        .eval('PV_LIAB_PRM = LIAB_PRM*DISC_FAC_PRM') \
        .eval('LOSS_ADJ_PRM = LIAB_PRM*LOSS_ADJ_RATE_PRM')
자동차_보험료부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
자동차_보험료부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
자동차_보험료부채_집계 = 자동차_보험료부채.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])[['PV_LIAB_PRM', 'LOSS_ADJ_PRM']].sum().reset_index()

## 준비금부채
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'CRD_GRD', 'BOZ_CD']
자동차_개별추산액_집계 = 자동차_개별추산액_가공.groupby(KEYS)['OST_AMT'].sum().astype(float).reset_index()
자동차_IBNR_집계 = 자동차_IBNR_가공.groupby(KEYS, as_index=False)['IBNR'].sum()
자동차_준비금부채 = 자동차_개별추산액_집계 \
    .merge(자동차_IBNR_집계, on=KEYS, how='left') \
    .merge(자동차_손해조사비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .assign(OST_AMT = lambda x: x['OST_AMT'].fillna(0)) \
    .assign(IBNR = lambda x: x['IBNR'].fillna(0)) \
    .assign(LAE_RATIO = lambda x: x['LAE_RATIO'].fillna(0)) \
    .merge(자동차_할인요소[['BOZ_CD', 'DISC_FAC_RSV']], on=['BOZ_CD'], how='left') \
    .merge(손실조정율[['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_RSV']], on=['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD'], how='left') \
    .assign(LOSS_ADJ_RATE_RSV = lambda x: x['LOSS_ADJ_RATE_RSV'].fillna(0).astype(float)) \
    .eval('FUT_LAE = (OST_AMT*0.5+IBNR*1.0)*LAE_RATIO') \
    .eval('LIAB_RSV = OST_AMT+IBNR+FUT_LAE') \
    .eval('PV_LIAB_RSV = LIAB_RSV*DISC_FAC_RSV') \
    .eval('LOSS_ADJ_RSV = LIAB_RSV*LOSS_ADJ_RATE_RSV')
자동차_준비금부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
자동차_준비금부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
자동차_준비금부채_집계 = 자동차_준비금부채.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])[['PV_LIAB_RSV', 'LOSS_ADJ_RSV']].sum().reset_index()

In [16]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/자동차_현행추정부채_{now}.xlsx', 'xlsxwriter') as writer:
    자동차_보험료부채_집계.to_excel(writer, '자동차_보험료부채', index=False)
    자동차_준비금부채_집계.to_excel(writer, '자동차_준비금부채', index=False)